In [ ]:
# import necessary packages
import numpy as np
import pandas as pd

# path to data we just downloaded
# if you are interested in how modelpoints were created,
# check out the `generating_modelpoints.ipynb` file in the repo
DATA_XLS = "./dstoolkit/model_point.xlsx"

# ingesting our data into a dataframe (df)

### Creating & editting spreadsheets from CLI with `openpyxl` & 'xlwings`